In [107]:
import gym

In [108]:
import numpy as np


In [109]:
import random

In [110]:
import matplotlib.pyplot as plt

In [132]:
flipped_4X4 = ['FFFS',
        'HFHF',
        'HFFF',
        'GFFH'] 

In [133]:
env = gym.make('FrozenLakeNotSlippery-v0', is_slippery=False, desc = flipped_4X4)

# Instantiate the Environment.
# env = gym.make('FrozenLake-v0')

# To check all environments present in OpenAI
# print(envs.registry.all())

# visualize 4x4 frozen lake
env.render()


FFFS
HFHF
HFFF
GFFH


In [142]:
# Total number of States and Actions
n_states = env.observation_space.n
n_actions = env.action_space.n
n_rows = int(np.sqrt(n_states))
n_cols=int(np.sqrt(n_states))
print( "States = ", n_states)
print( "Actions = ", n_actions)
print( "Rows = ", n_rows)
print( "Cols = ", n_cols)

States =  16
Actions =  4
Rows =  4
Cols =  4


In [164]:
# Start with 80% random actions to explore the environment
# exploration decays with each timestep by a factor of 0.999 until it hits 10% exploration rate 

epsilon = 1.0
epsilon_final = 0.10
epsilon_decay = 0.9999

In [165]:
gamma = 0.90 # discount factor
learning_rate = 0.90 #how important is the difference between q-val from q-table and what's observed

In [166]:

#Q = np.random.rand(15,4)*(1/100)
#Q = np.vstack([Q, np.zeros((1,4))])

Q = np.array([[0.        , 0.47827059, 0.59049   , 0. ],
       [0.53144098, 0.        , 0.6561    , 0.        ],
       [0.5904841 , 0.729     , 0.59048622, 0.        ],
       [0.65609993, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.53144047],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.81      , 0.        , 0.65609999],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.81      , 0.        ],
       [0.72899264, 0.9       , 0.        , 0.72899271],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.72899788],
       [0.65601433, 0.        , 1.        , 0.80919   ],
       [0.        , 0.        , 0.        , 0.        ]])

In [167]:
##assign index to each state using state-matrix

state_matrix = np.arange(0,16).reshape(4,4)
state_matrix

def rowsandcols(state):
    ''' input: state returned by env
    output: location of state as (row,col) tuple'''
    return int(np.where(state_matrix ==state)[0]), int(np.where(state_matrix ==state)[1])

In [168]:
def choose_random_action(state):
    ''' input = state returned by env.
      output = a random action depending on 
      whether the state is an edge/ corner or landlocked tile '''

   ##corner points    
    if rowsandcols(state)==( 0,0):
        action = random.choice([1,2])
    elif rowsandcols(state) ==(0,n_cols):
        action = random.choice([0,1])
    elif rowsandcols(state)==(n_rows,0):
        action = random.choice([2,3])
    elif rowsandcols(state)==(n_rows,n_cols):
        action = random.choice([0,3])
    ## no action needed to be chosen for (3,3) -- terminal state 

    ##edge tiles
    elif rowsandcols(state)  == (0,1) or rowsandcols(state)  ==  (0,2):
        action = random.choice([0,1,2])
    elif rowsandcols(state) == (1,0) or rowsandcols(state)  == (2,0):
        action = random.choice([1,2,3])
    elif rowsandcols(state) == (1,3) or rowsandcols(state) == (2,3):
        action = random.choice([0,1,3])
    elif rowsandcols(state) == (3,1) or rowsandcols(state) == (3,2) :
        action = random.choice([0,2,3])
  
    else:
        action = env.action_space.sample()
    
    return action

In [169]:
num_episodes = 100000
steps_total = [] # store number of steps taken in each episode
rewards_total = [] #store reward obtained for each episode
epsilon_total = [] #store epsilon obtained at the end of each episode

In [170]:
for i_episode in range(num_episodes):
    
    # resets the environment
    state = env.reset()
    step = 0

    while True:
        
        step += 1
        
        
        ## generating a random num
        random_for_epsilon = np.random.rand()
        
        ## if random num lesser or equal to epsilon, then select random action
        if random_for_epsilon <= epsilon:     
            action = choose_random_action(state)  ##replace with action = env.action_space.sample() for action without restrictions
        
        ## else update q-vals for given state with infitismly small random nums and select argmax(a) for Q(s,a)
        else: 
            random_values = Q[state] + np.random.rand(1,n_actions)/100
            action =  np.argmax(random_values)
        
        
        ## as epsilon decays with more timesteps, the prob. of selecting a random val < e decays --> more likely to exploit.     
        if epsilon > epsilon_final:
            epsilon *= epsilon_decay
        
        ## env gives reward and next state and whether we've reached terminal state upon taking a at current state.. 
        new_state, reward, done, info = env.step(action)
        ##if you want reward penalized at for each timestep
        ## reward= reward*(0.9**step) 

        # filling the Q Table - 
        Q[state, action] = (1- learning_rate)*Q[state, action] + learning_rate*(reward + gamma * np.max(Q[new_state]))
        
        # Setting new state for next action
        state = new_state
        
        # env.render()
        
        if done:
            steps_total.append(step)
            rewards_total.append(reward)
            epsilon_total.append(epsilon)
            if i_episode % 10 == 0:
                print('Episode: {} Reward: {} Steps Taken: {}'.format(i_episode,reward, step, epsilon))
            break

Episode: 0 Reward: 0.0 Steps Taken: 8
Episode: 10 Reward: 0.0 Steps Taken: 6
Episode: 20 Reward: 0.0 Steps Taken: 4
Episode: 30 Reward: 0.0 Steps Taken: 10
Episode: 40 Reward: 0.0 Steps Taken: 10
Episode: 50 Reward: 0.0 Steps Taken: 7
Episode: 60 Reward: 0.0 Steps Taken: 10
Episode: 70 Reward: 0.0 Steps Taken: 8
Episode: 80 Reward: 0.0 Steps Taken: 4
Episode: 90 Reward: 0.0 Steps Taken: 2
Episode: 100 Reward: 0.0 Steps Taken: 16
Episode: 110 Reward: 0.0 Steps Taken: 7
Episode: 120 Reward: 0.0 Steps Taken: 7
Episode: 130 Reward: 1.0 Steps Taken: 11
Episode: 140 Reward: 0.0 Steps Taken: 5
Episode: 150 Reward: 0.0 Steps Taken: 4
Episode: 160 Reward: 0.0 Steps Taken: 2
Episode: 170 Reward: 0.0 Steps Taken: 5
Episode: 180 Reward: 0.0 Steps Taken: 3
Episode: 190 Reward: 0.0 Steps Taken: 6
Episode: 200 Reward: 0.0 Steps Taken: 4
Episode: 210 Reward: 0.0 Steps Taken: 14
Episode: 220 Reward: 0.0 Steps Taken: 3
Episode: 230 Reward: 0.0 Steps Taken: 7
Episode: 240 Reward: 0.0 Steps Taken: 5
Episo

In [ ]:
plt.axhline(y=6, color='green', linestyle='-')
plt.bar(np.arange(0, 100000), steps_total, 
        width = 0.4, color = 'blue')
plt.xlabel('episode index')
plt.ylabel('number of steps taken')
plt.title('4x4: restricted actions')

Text(0.5, 1.0, '4x4: restricted actions')

In [163]:
Q

array([[0.00000000e+00, 4.78296165e-01, 6.56093411e-01, 0.00000000e+00],
       [5.90271254e-01, 7.29000000e-01, 6.56100000e-01, 0.00000000e+00],
       [6.56100000e-01, 7.29000000e-01, 5.90490000e-01, 0.00000000e+00],
       [6.56100000e-01, 6.56100000e-01, 5.90490000e-01, 5.90490000e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.31440470e-01],
       [4.78296418e-01, 8.10000000e-01, 7.29000000e-01, 6.56099993e-01],
       [0.00000000e+00, 8.10000000e-01, 0.00000000e+00, 6.56099990e-01],
       [7.29000000e-01, 6.56100000e-01, 0.00000000e+00, 5.90490000e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.00000000e-01, 7.28999993e-01, 7.28991404e-01],
       [8.10000000e-01, 8.10000000e-01, 6.56099935e-01, 7.29000000e-01],
       [7.29000000e-01, 0.00000000e+00, 0.00000000e+00, 6.56100000e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 8.10000000e